In [1]:
import torch
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

c:\KM\Virtual ENV\venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader=PyMuPDFLoader('data/Private_Cred_Fin_Stab_Paper.pdf')
data=loader.load()
data[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2025-12-16T18:54:15+00:00',
 'source': 'data/Private_Cred_Fin_Stab_Paper.pdf',
 'file_path': 'data/Private_Cred_Fin_Stab_Paper.pdf',
 'total_pages': 56,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '2025-12-16T18:54:15+00:00',
 'trapped': '',
 'modDate': 'D:20251216185415Z',
 'creationDate': 'D:20251216185415Z',
 'page': 0}

In [3]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1200,chunk_overlap=200,separators=['\n\n','\n','.',' '])
chunks=text_splitter.split_documents(data)
len(chunks)

143

In [4]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vector_db=FAISS.from_documents(chunks,embeddings)
vector_db.save_local('RAG INDEX')

In [5]:
MODEL='Qwen/Qwen2.5-1.5B-Instruct'
tokenizer=AutoTokenizer.from_pretrained(MODEL)
model=AutoModelForCausalLM.from_pretrained(MODEL,device_map='auto',dtype=torch.bfloat16,low_cpu_mem_usage=True)
pipe=pipeline(task='text-generation',temperature=0.2,do_sample=True,tokenizer=tokenizer,model=model,max_new_tokens=512,repetition_penalty=1.2,
              no_repeat_ngram_size=3)

Device set to use cuda:0


In [6]:
def format_prompt(question):
    search_resluts=vector_db.similarity_search(question,k=5)
    context='\n---\n'.join([doc.page_content for doc in search_resluts])
    prompt=f'''You are an experienced Financial Analyst.You must answer the question based on the context provided
context:
{context}

question:{question}
Answer: '''
    response=pipe(prompt,return_full_text=False)
    return response[0]['generated_text']

In [7]:
print(format_prompt('Summarize this Document'))

 The document appears to be discussing various aspects related to Business Development Companies (BDCs) within the finance sector. Here's a summary:

- **Data Breakdown**: 
    - This section breaks down how businesses known as "Business Development Companies" (BDOCS) manage their finances during periods when they're trying to reduce debts ("deleverage").
    
- **Decile Analysis**:
    - Different groups of BDOCs were analyzed according to their Initial Asset Ratio (ACR).
        * Bottom Decile: 28.5%-29%
        * Middle Group: ~30%-31%
        + Top Decile < 3%

- **Risk Factors**:
   - Various factors affecting these companies' ability to leverage funds included Industry Default Beta.

- **Financial Returns**:
     - An analysis was conducted regarding potential severe market scenarios where stock prices might drop drastically compared to baseline levels.

- Data Sources & Methods:
    * Information comes primarily from public reports like annual statements (SEC filings) and pitch

In [8]:
print(format_prompt('What is BDC and also What are BDC assets?'))

 BDC stands for Business Development Companies, which refers to privately held investment vehicles designed specifically for providing funding to smaller businesses through various types of loans including mortgages, equipment finance, commercial paper, etc., while adhering strictly to regulatory guidelines set forth by the Securities Exchange Commission (SEC).

Regarding your second part regarding what constitutes "BDC Assets," according to the information you've shared:

"BDCs' assets include both principal amounts borrowed against collateralized receivables ("loans") and other securities issued by third parties." 

So essentially, BCD's main assets consist of two parts - first, any money lent out to borrowers who agree to repay them over time; this includes things like mortgage-backed bonds, auto loans, business lines of credit, among others.

Secondly, BDOs hold onto those same debts until repayment occurs, plus additional cash reserves called 'buffers'. These buffer levels vary be

In [9]:
print(format_prompt('What information does the summary statistics table of 2024Q4 provide?'))

 Based on Table A2 which provides regression coefficients between various financial metrics including Industry Default Rate and Aggregate Default Rate, it shows how much variation or volatility there is among these two variables within specific time periods.

For instance, Aerospace & Defence has an "Industry Default" Beta coefficient of.89 indicating this sector's risk level compared to overall market risks; Automotive also exhibits high sensitivity (.86), suggesting similar levels of vulnerability but slightly lower exposure when measured against broader economic conditions.

These figures help investors understand potential correlations they might face across sectors due to commonalities in creditworthiness trends observed through historical data spanning multiple years up till 24 quarters ago - crucial insights before making investment decisions involving diverse portfolios. 

It highlights key areas where certain investments may be less risky given current macroeconomic indicators

In [10]:
print(format_prompt('What are the portfolio holdings of PitchBook'))

 Portfolio holdings data includes details like issuer name,industry,fair value,amortised cost,number of shares,maturity dates,and terms of loans.It covers both publicly traded bdc's and jvs.

Based on this passage we can infer what pitchbook does:

Pitchbook obtains detailed portfolio holdings information through partnerships with companies offering access to corporate finance services. They collect comprehensive datasets covering aspects such as issuers' characteristics, valuation metrics, investment amounts, maturities, and specific contractual agreements associated with individual investments within a diversified set of sectors. These insights enable them to offer valuable analysis tools tailored specifically towards understanding complex capital markets dynamics involving large-scale institutional investors who manage substantial pools of money dedicated toward diverse business opportunities. 

The key points highlighted here emphasize the importance of having accurate and up-to-da

In [11]:
print(format_prompt('What is DRIP?What Role does DRIP play in this document?'))

 Dividends can often be automatically reinvested into additional shares or units through what's known as a "dividend reinvestment plan" (DRIp), commonly referred to simply as a “DRIP.” This allows shareholders who hold stocks directly rather than indirectly via mutual funds or ETFs to participate in any future share price appreciation without paying taxes on those gains until sold.

In financial analysis documents like this one, DRIPS serve several important roles:

- **Tax Efficiency**: They provide tax benefits because no income tax needs to be paid upon receiving the distributed earnings; instead, you get them free-of-cost within your investment holdings.
  
- **Economic Impact**: When combined with market conditions, particularly during periods of economic downturns, DRIsPs help mitigate losses by allowing owners to benefit financially even if there isn't immediate growth due to reduced trading activity.

- **Market Participation**: Through automatic re-investment, it encourages pa

In [12]:
print(format_prompt('What is RIC? How much net income does BDC distribute?'))

 According to the given text, "We define a RIC as a Closed-End Fund that invests primarily in privately held businesses." Therefore, RIC stands for 'Registered Investment Company'. Regarding your second query about how much net profit BDC distributes, according to Equation 2 mentioned earlier:

\[ \text{FCF} = \text{{Net Income}} -  {0. 9}\times\text{{net income}}-\text{{Pik Interest}}+\text{{Principal Payments}}-{R}{e}{d}{m}{i}{t}{ions}-{D}{e{\_}b{e}{t}s {\_} Maturity } \]

The equation states that Free Cash Flow (\(\text{ FCFF }\)) equals Net Income minus 9 times its own amount plus principal payments less redemption amounts and maturity debts.

So, while there's no specific numerical distribution stated here without additional calculations, this formula indicates that most of the company’s profits go towards paying back borrowed money rather than distributing excess earnings among investors through dividends. 

In summary, RICS like BDC typically retain significant portions of their